In the transformer section we always used **.fit(dataset).transform(dataset)** for every transformer. This seems quite redundant, why not put all transformers together and just call it once? That is the role of pipeline.

#### Pipeline is collection of transformers

Apart from simplifying the code it adds greatly to reusability.
 - Imagine we are preparing data for model training and we train the model on January dataset.
 - Now February ends and we want to score the new dataset.
     - The new data are unprepared
     - Without pipeline we would need to again prepare them for the model
     - But with pipeline we just save pipeline during training and when it is time to score, we load it.
     - Whole data preparation becomes just two lines

## 3.1 How can we create a pipeline?
Pipeline creation is simple. At begining we create a Python list with transformers in order and pipeline is then created with just one line.

**Example:** We take categorical value HouseStyle, we want to string-index it and on-hot-encode it.

First we create session and load the data:

In [0]:
input_data = spark.read.parquet("/FileStore/cleanData/")

In [0]:
int_c = [c for c in input_data.columns]

Now lets see the normal way of doing this without a pipeline:

In [0]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol = "HouseStyle"
                            ,outputCol='HouseStyle_index'
                            ,handleInvalid="keep")

from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols = ["HouseStyle_index"],
                                 outputCols=["HouseStyle_index_vec"])

data_indexed = indexer.fit(input_data).transform(input_data)
data_encoded = encoder.fit(data_indexed).transform(data_indexed)

data_encoded.select("Id", "HouseStyle", "HouseStyle_index","HouseStyle_index_vec").show(10)

+----+----------+----------------+--------------------+
 Id|HouseStyle|HouseStyle_index|HouseStyle_index_vec|
+----+----------+----------------+--------------------+
 496| 1Story| 0.0| (8,[0],[1.0])|
 917| 1Story| 0.0| (8,[0],[1.0])|
 969| 1.5Fin| 2.0| (8,[2],[1.0])|
 534| 1Story| 0.0| (8,[0],[1.0])|
 31| 2Story| 1.0| (8,[1],[1.0])|
 711| 1Story| 0.0| (8,[0],[1.0])|
1338| 1Story| 0.0| (8,[0],[1.0])|
 706| 2Story| 1.0| (8,[1],[1.0])|
1326| 1Story| 0.0| (8,[0],[1.0])|
 813| 1Story| 0.0| (8,[0],[1.0])|
+----+----------+----------------+--------------------+
only showing top 10 rows

Now let's do it with a pipeline that will look like this:

In [0]:
from pyspark.ml import Pipeline

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol = "HouseStyle"
                            ,outputCol='HouseStyle_index'
                            ,handleInvalid="keep")

from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols = ["HouseStyle_index"],
                                 outputCols=["HouseStyle_index_vec"])

my_pipeline = Pipeline(stages = [indexer, encoder])
data_transformed = my_pipeline.fit(input_data).transform(input_data)

data_transformed.select("Id", "HouseStyle", "HouseStyle_index","HouseStyle_index_vec").show(10)

+----+----------+----------------+--------------------+
 Id|HouseStyle|HouseStyle_index|HouseStyle_index_vec|
+----+----------+----------------+--------------------+
 496| 1Story| 0.0| (8,[0],[1.0])|
 917| 1Story| 0.0| (8,[0],[1.0])|
 969| 1.5Fin| 2.0| (8,[2],[1.0])|
 534| 1Story| 0.0| (8,[0],[1.0])|
 31| 2Story| 1.0| (8,[1],[1.0])|
 711| 1Story| 0.0| (8,[0],[1.0])|
1338| 1Story| 0.0| (8,[0],[1.0])|
 706| 2Story| 1.0| (8,[1],[1.0])|
1326| 1Story| 0.0| (8,[0],[1.0])|
 813| 1Story| 0.0| (8,[0],[1.0])|
+----+----------+----------------+--------------------+
only showing top 10 rows

## 3.2 Creating whole pipeline for our data
Now we know what pipeline is. We can combine it with transformers from previous section and create whole pipeline for our dataset.

In [0]:
pipeline_stages = []

numericalFeatures = ['LotArea', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', 'GrLivArea', 'GarageArea', 'OpenPorchSF']
categoricalFeatures = ['MSSubClass', 'MSZoning', 'HouseStyle', 'OverallQual', 'OverallCond', 'ExterQual', 'Foundation', 
                       'BsmtQual', 'BsmtCond', 'HeatingQC', 'CentralAir', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces', 
                       'GarageCars', 'GarageQual', 'PoolQC', 'Neighborhood']

from pyspark.ml import Pipeline
from pyspark.ml.feature import SQLTransformer, OneHotEncoder, StringIndexer, VectorAssembler

# Remove outliers
sqlTransformer = SQLTransformer()\
  .setStatement("""
      SELECT
        *
        , case 
            when LotArea < 1450.0 then 1450.0 
            when LotArea > 17690.0 then 17690.0 
            else LotArea 
          end LotArea_bound
        , case 
            when YearBuilt < 1885.0 then 1885.0 
            when YearBuilt > 2069.0 then 2069.0 
            else YearBuilt 
          end YearBuilt_bound
        , case 
            when YearRemodAdd < 1911.5 then 1911.5 
            when YearRemodAdd > 2059.5 then 2059.5 
            else YearRemodAdd 
          end YearRemodAdd_bound
        , case 
            when TotalBsmtSF < 40.5 then 40.5
            when TotalBsmtSF > 2052.5 then 2052.5 
            else TotalBsmtSF 
          end TotalBsmtSF_bound
        , case 
            when GrLivArea < 156.0 then 156.0 
            when GrLivArea > 2748.0 then 2748.0 
            else GrLivArea 
          end GrLivArea_bound
        , case 
            when GarageArea < -39.0 then -39.0 
            when GarageArea > 945.0 then 945.0 
            else GarageArea 
          end GarageArea_bound
        , case 
            when OpenPorchSF < -102.0 then -102.0 
            when OpenPorchSF > 170.0 then 170.0 
            else OpenPorchSF 
          end OpenPorchSF_bound
        , cast(
              case 
                when SalePrice < 3750.0 then 3750.0
                when SalePrice > 340150.0 then 340150.0 
                else SalePrice 
               end as double) SalePrice_bound
        , SalePrice label
      FROM __THIS__
    """)

pipeline_stages.append(sqlTransformer)

# Index all categorical features
for c in categoricalFeatures:
    indexer = StringIndexer(inputCol = c ,outputCol = c + "_index", handleInvalid = "keep")
    pipeline_stages.append(indexer)

# One-hot encode all indexed variables
encoder = OneHotEncoder(inputCols = [(x + "_index") for x in categoricalFeatures],
                                 outputCols=[(x + "_index_vec") for x in categoricalFeatures])

pipeline_stages.append(encoder)

# Assemble one-hot encoded variables with numerical features with removed outliers
assembler = VectorAssembler(inputCols=[(x + "_index_vec") for x in categoricalFeatures] 
                                + [(x + "_bound") for x in numericalFeatures], 
                            outputCol="features")

pipeline_stages.append(assembler)


In [0]:
pipeline_stages

Out[6]: [SQLTransformer_f039b1decd7c,
 StringIndexer_6e28dfa965e5,
 StringIndexer_06ae9605e45c,
 StringIndexer_1c7775d43803,
 StringIndexer_0b1a9955aaa2,
 StringIndexer_55a703988fdc,
 StringIndexer_8249c6c174f4,
 StringIndexer_c12adc4076b5,
 StringIndexer_e0fbb89523a0,
 StringIndexer_00ff675e0c2c,
 StringIndexer_0611cde6c9d7,
 StringIndexer_95fb33dc535d,
 StringIndexer_9f2ffd0c4c72,
 StringIndexer_8524ae4ad5f3,
 StringIndexer_8b1bd8245dce,
 StringIndexer_989c4c6f0a05,
 StringIndexer_087e79f224fd,
 StringIndexer_940c0464344c,
 StringIndexer_4fe0019ad55f,
 OneHotEncoder_58bd37f98a5d,
 VectorAssembler_ac0978ef3d68]

In [0]:
# Create pipeline and fit it
pipeline = Pipeline(stages = pipeline_stages)
final_data = pipeline \
                .fit(input_data) \
                .transform(input_data) \
                .select("Id", "label","features")

final_data.write.mode("overwrite").parquet("/wsp/wsp_crm/final_data")

final_data.show(1, False)

+---+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
Id |label|features |
+---+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
496|34900|(141,[4,19,20,32,38,47,52,57,62,68,73,74,82,90,97,100,105,124,134,135,136,137,138,140],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7879.0,1920.0,1950.0,720.0,720.0,170.0])|
+---+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row

In [0]:
input_data.printSchema()

root
-- Id: integer (nullable = true)
-- SalePrice: integer (nullable = true)
-- LotArea: integer (nullable = true)
-- YearBuilt: integer (nullable = true)
-- YearRemodAdd: integer (nullable = true)
-- TotalBsmtSF: integer (nullable = true)
-- GrLivArea: integer (nullable = true)
-- GarageArea: integer (nullable = true)
-- OpenPorchSF: integer (nullable = true)
-- MSSubClass: integer (nullable = true)
-- MSZoning: string (nullable = true)
-- HouseStyle: string (nullable = true)
-- OverallQual: integer (nullable = true)
-- OverallCond: integer (nullable = true)
-- ExterQual: string (nullable = true)
-- Foundation: string (nullable = true)
-- BsmtQual: string (nullable = true)
-- BsmtCond: string (nullable = true)
-- HeatingQC: string (nullable = true)
-- CentralAir: string (nullable = true)
-- KitchenQual: string (nullable = true)
-- TotRmsAbvGrd: integer (nullable = true)
-- Fireplaces: integer (nullable = true)
-- GarageCars: integer (nullable = true)
-- GarageQual: string (nullable = true)
-- PoolQC: string (nullable = true)
-- Neighborhood: string (nullable = true)
-- PriceNtile: double (nullable = true)

## <font color='orange'>3.3 Linear Regression</font>
We have prepared data for the model training. We can now use them to train a simple Linear regression.

Basic info about Linear Regression:
* Predicts continuous variable
* Easy to interpret the results
* Sensitive to outliers
* **Documentation:** https://spark.apache.org/docs/2.3.0/ml-classification-regression.html#linear-regression 

Predicted variable is a linear equation:
## $$ y_i =\beta_0+\beta_1x_1 ... $$

First we define the model, lets use default parameters.

In [0]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='label')

Models are **transformers** too, so to train we call **.fit(dataset)**

In [0]:
lr_model = lr.fit(final_data)

To score dateset we call **.transform**. Normally we would split dataset on test/train, right now we just use one dataset for training and scoring:

In [0]:
lr_scored = lr_model.transform(final_data)

In [0]:
lr_scored.show(10)

+----+-----+--------------------+------------------+
 Id|label| features| prediction|
+----+-----+--------------------+------------------+
 496|34900|(141,[4,19,20,32,...| 41995.62846823002|
 917|35311|(141,[0,19,20,36,...| 40557.4601901382|
 969|37900|(141,[2,16,22,34,...| 74030.03398823785|
 534|39300|(141,[0,15,20,37,...| 39306.24361327756|
 31|40000|(141,[6,19,21,32,...|59234.935649413615|
 711|52000|(141,[4,15,20,34,...| 75075.31777142885|
1338|52500|(141,[4,16,20,32,...| 51286.16704731935|
 706|55000|(141,[9,16,21,32,...| 69691.30731086445|
1326|55000|(141,[4,16,20,32,...| 52818.99424609321|
 813|55993|(141,[0,19,20,28,...| 71221.13923818001|
+----+-----+--------------------+------------------+
only showing top 10 rows

### 3.3.1 Model evaluation
With trained model we can evalute it's performance. Performance metric are different for regression/classification and even different for individual algorithms. Here we try to use the most common.

**Coefficients of equation + intercept**

In [0]:
lr_model.coefficients

Out[13]: DenseVector([1181.887, 13617.2619, 1326.9367, -8084.0936, -2276.6826, -8964.0555, 6294.8592, -14274.1189, -24204.1385, -3524.1059, -16846.9326, -27244.2742, 20658.4948, -19883.6955, 1454.172, -1048.9146, -2225.7464, 16098.2834, -672.5172, -32309.2347, -277.6146, -7395.1659, -317.8975, 24928.98, 22668.5249, -17124.5514, 19919.5874, 10959.0283, -11443.4352, -7361.6815, 2120.7124, 17675.6832, -10596.1035, 49671.611, -9892.3595, 81061.8374, 9822.5147, -8862.7219, -6892.2867, 4385.8756, 10380.0337, 13069.2121, -16998.9949, -28623.8637, 27954.0695, -22906.5648, -18722.4636, -89.7856, -872.8562, 6846.9328, -2085.1791, 801.8957, -435.343, -1067.8897, -88.4467, 9274.0529, -15285.7922, -4601.0998, -2199.6965, 18616.0454, 8516.6153, 2041.1463, 906.938, -1393.5403, -7665.1454, 8516.6153, -3817.3093, 478.592, -632.2212, -575.222, 3108.8745, -14866.7069, 492.1386, -492.1386, -2637.5295, -317.6924, 13068.3083, -3773.1784, -358.1519, -3473.9437, 2098.4727, -1270.757, 4166.7558, -6488.8562, 11834.4961, 3240.2605, 5729.5201, 12377.4124, -15468.7292, 1009.1689, -3307.4326, -1044.9804, 16696.6806, -37819.645, -2005.348, -6343.5524, 17433.8459, -5585.5838, 26487.6877, 1365.5735, -5585.5838, -1802.0519, 16550.029, 48374.9042, -15372.9734, -6864.1269, -62458.9097, 133683.3658, -16117.2037, -8566.985, 1082.6348, -12591.2665, -11887.9799, 1778.0838, -2119.2977, 20974.8226, -9001.3706, -11731.612, 25.0732, 1571.4921, 20172.644, -10294.3635, 40986.8279, 246.4437, -1437.1995, 6916.8725, -5000.1762, 47312.1167, 3865.2431, -8737.2396, 11579.4292, 18639.2375, 10934.9554, 9098.015, 1.6595, 367.7213, 119.8186, 35.2256, 47.1859, -3.063, 9.2885])

In [0]:
lr_model.intercept

Out[14]: -892719.8318628379

In [0]:
from itertools import chain
import pandas as pd

attrs = sorted((attr["idx"], attr["name"]) for attr in 
               (chain(*lr_scored.schema["features"].metadata["ml_attr"]["attrs"].values())
               ))

pairs = [(name, lr_model.coefficients[idx]) for idx, name in attrs]
sorted_pairs = sorted(pairs, key = lambda p: abs(p[1]), reverse=True)
variables = sorted_pairs

dataset = pd.DataFrame(variables, columns = ["predictors", "coefficients"]) 

display(dataset)

predictors,coefficients
PoolQC_index_vec_Ex,133683.36584856323
OverallQual_index_vec_10,81061.83740468683
PoolQC_index_vec_Gd,-62458.90973271046
OverallQual_index_vec_9,49671.61103936324
GarageQual_index_vec_Ex,48374.904216237686
Neighborhood_index_vec_3,47312.11666768819
Neighborhood_index_vec_1,40986.82788141409
Fireplaces_index_vec_3,-37819.64501421669
MSZoning_index_vec_C (all),-32309.234654199103
OverallCond_index_vec_3,-28623.86373697755


In [0]:
lr_model.intercept

Out[16]: -892719.8318628379

**RMSE** - RootMeanSquaredError
- The RMSE measures how much error there is between two datasets comparing a predicted value and an observed or known value.
        The smaller an RMSE value, the closer predicted and observed values are.

In [0]:
lr_model.summary.rootMeanSquaredError

Out[17]: 26302.01435753805

**r^2** - R squared
- The R2 ("R squared") or the coefficient of determination is a measure that shows how close the data are to the fitted regression line. This score will always be between 0 and a 100% (or 0 to 1 in this case), where 0% indicates that the model explains none of the variability of the response data around its mean, and 100% indicates the opposite: it explains all the variability.
            That means that, in general, the higher the R-squared, the better the model fits our data.

In [0]:
lr_model.summary.r2

Out[18]: 0.8903093151289097